In [1]:
import pandas as pd
from collections import defaultdict

In [4]:
DATASET_PATH = '../datasets/RC_2018-02-28.xz'

In [ ]:
df = pd.read_json(DATASET_PATH, lines=True, chunksize=100).read()
# not sure if this is necessary/detrimental
df = df[(df.body != '[deleted]') & (df.body != '[removed]')]

Possible sentiment features: SentiStrength, Vader-Sentiment, LIWC. Both SentiStrength and LIWC are proprietary. LIWC also can be used to give many different psychological and language dimensions (if only it weren't proprietary).

TODO: learned Naive Bayes classifier

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def vader_features(analyzer, body):
    vs = analyzer.polarity_scores(body)
    return {'vad_'+k: v for k, v in vs.items()}

Readability features: word count, avg sentence length, avg word length, Gunning Fog, SMOG, Flesch-Kincaid. LIWC can give language features, but again is proprietary.

TODO: COCA fluency

In [ ]:
import textstat

def read_features(body):
    d = defaultdict(float)
    d['WC'] = textstat.lexicon_count(body)
    d['WPS'] = textstat.avg_sentence_length(body)
    d['WL'] = textstat.avg_letter_per_word(body)
    d['GI'] = textstat.gunning_fog(body)
    d['SMOG'] = textstat.smog_index(body)
    d['FK'] = textstat.flesch_kincaid_grade(body)
    return d

In [ ]:
import string
from collections import Counter

exclude = list(string.punctuation)

def ttr_feature(body):
    words = ''.join(ch for ch in body if ch not in exclude).split()
    type_count = Counter(words)
    return {'ttr': len(type_count) / float(sum(type_count.values()))}

In [ ]:
def all_features(comment):
    features = defaultdict(float)

    analyzer = SentimentIntensityAnalyzer()
    features.update(vader_features(analyzer, comment.body))
    
    features.update(read_features(comment.body))
    features.update(ttr_feature(comment.body))

    return features

In [ ]:
for i in range(5):
    f = all_features(df[i])
    print(f)

TODO: featurize each comment, make list of dicts, use sklearn.feature_extraction.DictVectorizer